<a href="https://colab.research.google.com/github/yuri-kasahara/Workshop-Tekst-som-data/blob/main/Example_scraper_NIBR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This is a script with an example of a scraper to obtain a list of hearings held by the Norwegian government with their respective comments.**

The two cells below will load the packages you need to run the scraper.

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.1wEyagwE8c/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.bMJSEYnb29/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.7gy2C1XBh3/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium


import regex as re
import requests
import time
import pandas as pd
import os
import logging
from bs4 import BeautifulSoup as BS
import urllib.request

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service


options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-blink-features=AutomationControlled')

service_driver = Service(executable_path=r'/usr/bin/chromedriver')



Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://deb.debian.org/debian-security buster/updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Now, let's get some data!

In [ ]:
 ## This is a link for one of the pages containing the list of reports published by NIBR.

url = "https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0"

driver.get(url)
html = driver.page_source

## You can use the developer tools of your web browser to help you identify the areas you want to scrape. You will probably also have to introduce some loops to properly separate each entry you want.
soup = BS(html, "html.parser")
link_reports = soup.find_all('h4', {'class':'artifact-title'})


print(link_reports)


[<h4 class="artifact-title">
<a href="/oda-xmlui/handle/11250/3103781">Risikovurdering og reintegrering av radikaliserte individer i Norden</a><span class="Z3988" title="ctx_ver=Z39.88-2004&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Adc&amp;rft_id=978-82-8309-391-9&amp;rft_id=1502-9794&amp;rft_id=https%3A%2F%2Fhdl.handle.net%2F11250%2F3103781&amp;rfr_id=info%3Asid%2Fdspace.org%3Arepository&amp;">
                    ﻿ 
                </span>
</h4>, <h4 class="artifact-title">
<a href="/oda-xmlui/handle/11250/3085164">Midtvegsevaluering av Program for folkehelsearbeid i Vestland</a><span class="Z3988" title="ctx_ver=Z39.88-2004&amp;rft_val_fmt=info%3Aofi%2Ffmt%3Akev%3Amtx%3Adc&amp;rft_id=978-82-8309-385-8&amp;rft_id=https%3A%2F%2Fhdl.handle.net%2F11250%2F3085164&amp;rfr_id=info%3Asid%2Fdspace.org%3Arepository&amp;">
                    ﻿ 
                </span>
</h4>, <h4 class="artifact-title">
<a href="/oda-xmlui/handle/11250/3054721">Flyktningers møte med NAV.  Kommunikasjon og 

In [ ]:
## Let's make a list of links to cover all 569 reports

Links = []

sequence = list(range(0, 561, 20))

for s in sequence:
    url = "https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0"+str(s)
    Links.append(url)

print(Links)

['https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=00', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=020', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=040', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=060', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=080', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0100', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0120', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0140', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0160', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0180', 'https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=0200', 'https://oda.oslomet.no/oda-xmlui/han

In [ ]:
## We have first to create empty lists to hold our scraped content

Rapport_tittel = []
Url_report = []

## Then we have to ask load our 'driver' and tell it what we want to collect. Let's introduce a loop so we cover all links in our previous list.
driver = webdriver.Chrome(service=service_driver, options=options)

for l in Links:

    print("Getting information from: ", l)
    driver.get(l)
    html = driver.page_source

## You can use the developer tools of your web browser to help you identify the areas you want to scrape. You will probably also have to introduce some loops to properly separate each entry you want.
    soup = BS(html, "html.parser")
    link_reports = soup.find_all('h4', {'class':'artifact-title'})

    for r in link_reports:
# These lines give you the name of the hearing
        t = r.find('a')
        print(t)
        Rapport_tittel.append(t.text.strip())
# These lines give you the link for the webpage of the hearing
        u = r.find('a')['href']
        lenken = "oda.oslomet.no"+str(u)
        Url_report.append(lenken)

    time.sleep(3)

driver.quit()


# Then you have to put all the text collected in a dataframe format by turning the lists into variables.
Reports_list = {}
Reports_list['Rapport_tittel'] = Rapport_tittel
Reports_list['Url_report'] = Url_report

# Here you will make the dataframe
df = pd.DataFrame.from_dict(Reports_list, orient='columns')

print(df)

# Now save it on your drive or on your computer
df.to_csv('drive/MyDrive/WorkshopTSD/reports_list.csv', index=False)


Getting information from:  https://oda.oslomet.no/oda-xmlui/handle/11250/2740292/recent-submissions?offset=00
<a href="/oda-xmlui/handle/11250/3103781">Risikovurdering og reintegrering av radikaliserte individer i Norden</a>
<a href="/oda-xmlui/handle/11250/3085164">Midtvegsevaluering av Program for folkehelsearbeid i Vestland</a>
<a href="/oda-xmlui/handle/11250/3054721">Flyktningers møte med NAV.  Kommunikasjon og tilgjengelighet i en digital kontekst.</a>
<a href="/oda-xmlui/handle/11250/3050589">Vil gjerne vite mer: Kompetansebehov om negativ sosial kontroll og æresrelatert vold blant ansatte i opplærings- og integreringstjenestene</a>
<a href="/oda-xmlui/handle/11250/3045883">Evaluering av opptrappingsplanen mot vold og overgrep. Sluttrapport</a>
<a href="/oda-xmlui/handle/11250/3045754">Boligmarkedsanalyse: Fredrikstad mot 2035</a>
<a href="/oda-xmlui/handle/11250/3041340">Sosiokulturell stedsanalyse for Strømsø</a>
<a href="/oda-xmlui/handle/11250/3039118">Senterstrukturer, regi

In [ ]:
df_2 = pd.read_csv("drive/MyDrive/WorkshopTSD/reports_list.csv", sep=',', dtype='string')


pd.options.display.max_colwidth = 150

print(df_2)

                                                                                                                                 Rapport_tittel  \
0                                                                          Risikovurdering og reintegrering av radikaliserte individer i Norden   
1                                                                                 Midtvegsevaluering av Program for folkehelsearbeid i Vestland   
2                                                           Flyktningers møte med NAV.  Kommunikasjon og tilgjengelighet i en digital kontekst.   
3    Vil gjerne vite mer: Kompetansebehov om negativ sosial kontroll og æresrelatert vold blant ansatte i opplærings- og integreringstjenestene   
4                                                                           Evaluering av opptrappingsplanen mot vold og overgrep. Sluttrapport   
..                                                                                                                    

In [ ]:
URLs_reports= df_2['Url_report']
print(URLs_reports)

0          oda.oslomet.no/oda-xmlui/handle/11250/3103781
1          oda.oslomet.no/oda-xmlui/handle/11250/3085164
2          oda.oslomet.no/oda-xmlui/handle/11250/3054721
3          oda.oslomet.no/oda-xmlui/handle/11250/3050589
4          oda.oslomet.no/oda-xmlui/handle/11250/3045883
                             ...                        
564    oda.oslomet.no/oda-xmlui/handle/20.500.12199/1282
565    oda.oslomet.no/oda-xmlui/handle/20.500.12199/1281
566    oda.oslomet.no/oda-xmlui/handle/20.500.12199/1280
567    oda.oslomet.no/oda-xmlui/handle/20.500.12199/1279
568    oda.oslomet.no/oda-xmlui/handle/20.500.12199/1268
Name: Url_report, Length: 569, dtype: string


Now, let's get data on the individual contents of each report.

The scraper below will use the links from the previous dataset.

In [ ]:
Title = []
Abstract = []
Year = []

driver = webdriver.Chrome(service=service_driver, options=options)

for link in URLs_reports:
    print("Getting information from: ", link)
    driver.get("https://" + str(link))
    time.sleep(3)
    html = driver.page_source
    soup = BS(html, "html.parser")

    t = soup.find('h2', {'class':'page-header first-page-header'})
    Title.append(t.text.strip())

    try:
        abs = soup.find_all('div', {'class':'simple-item-view-description item-page-field-wrapper table'})
        pattern_found = False
        for a in abs:
            clean_a = a.text.strip()
            if clean_a.startswith("Sammendrag") or clean_a.startswith("Abstract"):
              clean_abs = re.sub(r'^(Sammendrag|Abstract)\n', '', clean_a)
              print(clean_abs)
              Abstract.append(clean_abs)
              pattern_found = True

        if not pattern_found:
            Abstract.append(None)
    except:
        Abstract.append(None)

    try:
        y = soup.find('div', {'class':'simple-item-view-date word-break item-page-field-wrapper table'})
        clean_y = y.text.strip()
        clean_y = re.sub(r'^Utgivelsesdato', '', clean_y)
        pattern = r'\b\d{4}\b'
        clean_y = re.search(pattern, clean_y)
        Year.append(clean_y.group())

    except:
        Year.append(None)
driver.quit()

Info_reports = {}

Info_reports['Title'] = Title
Info_reports['Abstract'] = Abstract
Info_reports['Year'] = Year

df3 = pd.DataFrame.from_dict(Info_reports, orient='columns')

df3.to_csv('drive/MyDrive/WorkshopTSD/Info_reports.csv', index=False)

print(df3)


Getting information from:  oda.oslomet.no/oda-xmlui/handle/11250/3103781
Rapporten belyser arbeidet med risikovurdering og reintegrering av radikaliserte individer i de fire nordiske landene, Danmark, Sverige, Norge og Finland. Vi har intervjuet praktikere og andre myndighetspersoner med særlig erfaring med denne typen arbeid i Norden, samt gjennomgått forskning og annen litteratur. På bakgrunn av dette belyser vi sentrale praksiser og tiltak innen risikovurderinger og reintegrering, samt vurderinger av hvordan disse praksisene og tiltakene fungerer. Vi peker på sentrale utfordringer, og kommer med anbefalinger som vil kunne forbedre arbeidet på disse to områdene.
Getting information from:  oda.oslomet.no/oda-xmlui/handle/11250/3085164
Rapporten er ei midtvegsevaluering av Program for folkehelsearbeid i Vestland, som er ein del av det nasjonale Program for folkehelsearbeid i kommunane 2017–2027. Evalueringa har form av ei kartleggingsundersøking om korleis programmet fungerer i prosjek